# Data Lake Example 10 - Upload files into Minio for Trino usage

## 0. Load libraries and common configuration

In [3]:
# Install necessary packages
!pip install boto3 certifi pandas pyarrow sqlalchemy trino sqlalchemy-trino

In [4]:
import boto3
from botocore.client import Config
import pandas as pd
import os
import ssl
import certifi
import sys
import warnings
warnings.filterwarnings('ignore')

#Some issues might appear (SSL verification error) with yhe client if python is not properly configured. 
# You might find this line useful to skip the error 
ssl._create_default_https_context = ssl._create_unverified_context


# MinIO server connection information
minio_url = 'https://s3api.scene.local'  # Replace with your MinIO instance URL
access_key = 'testuser'       # Replace with your actual access key
secret_key = 'testscene'       # Replace with your actual secret key


# Initialize a session using boto3
session = boto3.session.Session()

# Create a client with the MinIO server
# Add "verify=False" tothe list if you have troubles with SSL verification
s3_client = session.client(
    's3',
    verify=False,
    endpoint_url=minio_url,    
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key,
    config=Config(signature_version='s3v4'),
    region_name='us-east-1'  # You can choose any region name. Not applicable here
)
print("Libraries loaded successfully")

Libraries loaded successfully


## 1. Convert CSV file into Parquet for better performance and Upload it to the Data Lake

In [3]:
orig_file_path='data.csv'
file_path='data.parquet'
bucket_name = 'testbucket'     # Bucket to upload the file to
object_name = 'csv/data.parquet'    # Object name in the bucket (can be a path like 'folder/file.txt')


# Read CSV
df = pd.read_csv(orig_file_path)

# Convert to Parquet
df.to_parquet(file_path, engine='pyarrow', index=False)


# Upload the file
s3_client.upload_file(file_path, bucket_name, object_name)
print("Upload successful")

Upload successful


## 2. List all files from a bucket within a Data Lake

In [6]:
bucket_name = 'testbucket'     # Bucket to upload the file to
# List objects in the bucket
response = s3_client.list_objects(Bucket=bucket_name)

# Print each file name (key)
if 'Contents' in response:
    for file in response['Contents']:
        print(file['Key'])
else:
    print("No files found in the bucket.")

athens.png
athens.webm
csv/data.parquet


## 3. Connect to Trino

In [12]:
from sqlalchemy import create_engine, text

# Trino connection details
engine = create_engine(
    "trino://testuser@trinoui.scene.local:443/iceberg",
    connect_args={"http_scheme": "https", "verify": False}  # Add a default user
)

# Corrected CREATE SCHEMA query
create_schema_query = "CREATE SCHEMA IF NOT EXISTS test WITH (location = 's3a://testbucket')"

# Execute the query
with engine.connect() as conn:
    conn.execute(text(create_schema_query))
    print("Schema 'test' created or already exists.")



ProgrammingError: (trino.exceptions.TrinoUserError) TrinoUserError(type=USER_ERROR, name=CATALOG_NOT_FOUND, message="line 1:1: Catalog 'iceberg' not found", query_id=20241117_152935_00004_2sv2g)
[SQL: CREATE SCHEMA IF NOT EXISTS test WITH (location = 's3a://testbucket')]
(Background on this error at: https://sqlalche.me/e/20/f405)

## 4. Delete a file from the Data Lake

In [8]:
# Step 2: Create table
create_table_query = """
CREATE TABLE IF NOT EXISTS test.people (
    id INT,
    name VARCHAR,
    age INT
)
WITH (
    format = 'PARQUET',
    external_location = 's3a://testbucket/csv/csv.parquet'
);
"""

# Step 3: Query the table
select_query = "SELECT * FROM test.people;"

# Execute the queries
with engine.connect() as conn:
    # Create schema
    conn.execute(text(create_schema_query))
    print("Schema 'test' created or already exists.")

    # Create table
    conn.execute(text(create_table_query))
    print("Table 'people' created or already exists.")

    # Query the table
    result = conn.execute(text(select_query))
    for row in result:
        print(row)


Delete successful
